In [9]:
import os.path as osp 

from trifinger_simulation.tasks import move_cube
from rrc.env import make_env, cube_env, initializers, termination_fns
from rrc_iprl_package.control.control_policy import (
    CuboidImpedanceControllerPolicy,
    ImpedanceControllerPolicy,
)
import numpy as np

from rrc_iprl_package.control.controller_utils_cube import OBJ_HALF_SIZE

In [14]:
difficulty = 4
init = initializers.centered_init(difficulty)
goal = init.get_goal()
goal.to_dict()

{'position': array([ 0.12211779, -0.01977259,  0.08001031]),
 'orientation': array([ 0.42404735, -0.80616657,  0.39928876, -0.10415273])}

In [10]:
FRAMESKIP = 1
MAX_STEPS = 120 * 1000 / 4
EP_LEN = None

difficulty = 1
init = initializers.centered_init(difficulty)
goal_pose = init.get_goal()
initial_pose = move_cube.sample_goal(-1)
object_shape = 'cube'
ep_len = EP_LEN or MAX_STEPS
if difficulty == 4:
    term_fn = termination_fns.stay_close_to_goal_level_4
else:
    term_fn = termination_fns.stay_close_to_goal
path = "./output" if osp.exists("./output") else None
    
env = cube_env.RobotCubeEnv(
    goal_pose.to_dict(),
    difficulty,
    action_type=cube_env.ActionType.TORQUE,
    initializer=init,
    frameskip=FRAMESKIP,
    episode_length=ep_len,
    termination_fn=term_fn,
    sim=True,
    visualization=True,
    path=path,
    return_timestamp=True,
    object_shape=object_shape,
)
goal = goal_pose.to_dict()
goal['position'] = goal['position'] - np.array([0, 0, OBJ_HALF_SIZE])
policy = ImpedanceControllerPolicy(
    env.action_space,
    initial_pose,
    goal,
    difficulty=difficulty,
    save_path=path,
    ycb=(object_shape == "ycb"),
)

observation = env.reset()
policy.set_init_goal(
    move_cube.Pose.from_dict(observation["achieved_goal"]),
    move_cube.Pose.from_dict(observation["desired_goal"]),
)
policy.reset_policy(observation, env.platform)

accumulated_reward = 0
is_done = False
old_mode = policy.mode
steps_so_far = 0
try:
    while not is_done:
        if MAX_STEPS is not None and steps_so_far == MAX_STEPS:
            break
        action = policy.predict(observation)
        observation, reward, is_done, info = env.step(action)
        if old_mode != policy.mode:
            print("mode changed: {} to {}".format(old_mode, policy.mode))
            old_mode = policy.mode
        # print("reward:", reward)
        accumulated_reward += reward
        steps_so_far += env.frameskip

except Exception as e:
    print("Error encounted: {}. Saving logs and exiting".format(e))
    env.save_custom_logs()
    policy.save_log()
    raise e

/scr-ssd/ksrini/trifinger_simulation/python/trifinger_simulation/trifinger_platform.py:310: UserWarning: Cameras are not enabled, so images in the camera observation are not initialized.  Create `TriFingerPlatform` with `enable_cameras=True` to get rendered camera images.
  "Cameras are not enabled, so images in the camera observation"


mode changed: TrajMode.PRE_TRAJ_LOWER to TrajMode.PRE_TRAJ_REACH
mode changed: TrajMode.PRE_TRAJ_REACH to TrajMode.REPOSITION
Error encounted: Not connected to physics server.. Saving logs and exiting


error: Not connected to physics server.